<a href="https://colab.research.google.com/github/AarkeTripathi/spaceship-titanic/blob/master/space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re

In [ ]:
train_data=pd.read_csv("train.csv")
train_data.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [ ]:
train_data['CryoSleep']=train_data['CryoSleep'].astype(bool)
train_data['VIP']=train_data['VIP'].astype(bool)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8693 non-null   bool   
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(3), float64(6), object(5)
memory usage: 772.6+ KB


In [ ]:
train_data['Destination'].value_counts()

TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64

In [158]:
f_train_data=train_data.drop(['PassengerId','Name'], axis=1)
f_train_data=f_train_data.dropna()
f_train_data.shape

(7111, 12)

In [159]:
f_train_data['Deck']=f_train_data['Cabin'].apply(lambda x: x.split('/')[0])
f_train_data['Side']=f_train_data['Cabin'].apply(lambda x: x.split('/')[2])

In [160]:
encode=LabelEncoder()
f_train_data['Deck']=encode.fit_transform(f_train_data['Deck'])
f_train_data['Side']=encode.fit_transform(f_train_data['Side'])

In [161]:
for column in f_train_data.columns:
  if f_train_data[column].dtype=='bool':
    f_train_data[column]=encode.fit_transform(f_train_data[column])

In [162]:
f_train_data.drop(['Cabin'],axis=1,inplace=True)

In [163]:
f_train_data.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
0,Europa,0,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,Earth,0,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,5,1
2,Europa,0,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,0,1
3,Europa,0,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,0,1
4,Earth,0,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,5,1


In [135]:
flt=pd.DataFrame([[] for i in range(f_train_data.shape[0])])
for column in f_train_data.columns:
  if f_train_data[column].dtype=='float64':
    flt[column]=f_train_data[column]
flt.dropna(inplace=True)

In [157]:
columns=flt.columns
scaler=StandardScaler()
flt=pd.DataFrame(scaler.fit_transform(flt), columns=columns)
flt.shape

(5781, 6)

In [139]:
flt.head(10)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.685716,-0.350624,-0.282927,-0.316173,-0.277149,-0.267736
1,-0.338782,-0.180597,-0.277641,-0.271300,0.201469,-0.228755
2,1.983413,-0.283549,1.817505,-0.316173,5.576982,-0.224326
3,0.275917,-0.350624,0.470668,0.349743,2.625070,-0.096753
4,-0.885181,0.122020,-0.241811,-0.045140,0.215417,-0.265964
5,1.027215,-0.350624,0.000772,-0.316173,-0.023455,-0.267736
6,-0.202182,-0.285109,0.621034,-0.310789,-0.277149,-0.267736
7,0.412516,-0.350624,0.178158,-0.285660,-0.088840,-0.267736
8,-1.021780,-0.350624,-0.282927,-0.316173,-0.277149,-0.267736
9,1.095515,-0.289788,4.001931,0.741036,-0.181251,-0.157882


In [140]:
for column in flt.columns:
  f_train_data[column]=flt[column]

In [156]:
f_train_data.Age.values

array([ 0.68571587, -0.33878192,  1.98341308, ...,         nan,
               nan,         nan])

In [146]:
f_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7111 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    7111 non-null   int64  
 1   CryoSleep     7111 non-null   int64  
 2   Destination   7111 non-null   int64  
 3   Age           4699 non-null   float64
 4   VIP           7111 non-null   int64  
 5   RoomService   4699 non-null   float64
 6   FoodCourt     4699 non-null   float64
 7   ShoppingMall  4699 non-null   float64
 8   Spa           4699 non-null   float64
 9   VRDeck        4699 non-null   float64
 10  Transported   7111 non-null   int64  
 11  Deck          7111 non-null   int64  
 12  Side          7111 non-null   int64  
dtypes: float64(6), int64(7)
memory usage: 1.0 MB


In [ ]:
vectorizer=TfidfVectorizer()

In [ ]:
lst=[]
for column in f_train_data.columns:
  if f_train_data[column].dtype=='O':
    f_train_data[column]=f_train_data[column].apply(lambda x: re.sub("[^a-zA-Z]", "", x))
    vectorizer.fit(f_train_data[column].values)
    vector=vectorizer.transform(f_train_data[column])
    vector=vector.toarray()
    lst.append(vector)   
lst=np.array(lst) 

In [ ]:
arr=[]
for i in range(lst.shape[0]):
  sub_arr=[]
  for j in range(lst.shape[1]):
    val=''
    for k in range(lst.shape[2]):
      val=val+str(int(lst[i][j][k]))
    sub_arr.append(int(val))
  arr.append(sub_arr)

In [ ]:
for sub_arr in range(len(arr)):
  arr[sub_arr]=[int(j) for j in arr[sub_arr]]

In [101]:
f_train_data.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
0,10,False,1,39.0,False,0.0,0.0,0.0,0.0,0.0,False,1,0
1,100,False,1,24.0,False,109.0,9.0,25.0,549.0,44.0,True,5,1
2,10,False,1,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0,1
3,10,False,1,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0,1
4,100,False,1,16.0,False,303.0,70.0,151.0,565.0,2.0,True,5,1


In [ ]:
cout=0
while(cout<len(arr)):
  for column in f_train_data.columns:
    if f_train_data[column].dtype=='O':
      f_train_data[column]=arr[cout]
      break
  cout=cout+1

In [ ]:
f_train_data['Destination'].value_counts()

1      4924
100    1515
10      672
Name: Destination, dtype: int64

In [142]:
x=f_train_data.drop('Transported',axis=1)
y=f_train_data['Transported']

In [143]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [144]:
model=RandomForestClassifier()

In [145]:
model.fit(x_train,y_train)

ValueError: ignored

In [111]:
model.score(x_test,y_test)

0.7947997189037245

In [114]:
from xgboost import XGBClassifier
model = XGBClassifier()

In [115]:
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.797610681658468